In [ ]:
from clickhouse_driver import Client

In [ ]:
client = Client(
    host='localhost',
    port=9000,
    database='fleet',
    user='fleet',
    password='fleet',
    settings={'use_numpy': True},
)

In [ ]:
time_position, longitude, latitude = client.execute(
    'select time_position, longitude, latitude '
    'from clean_states '
    'order by time_position desc '
    'limit 10_000_000',
    columnar=True,
)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'time_position': time_position, 'longitude': longitude, 'latitude': latitude})

In [ ]:
df.head()

In [ ]:
import datashader as ds
import colorcet as cc

In [ ]:

agg = ds.Canvas().points(df, 'longitude', 'latitude')
ds.tf.set_background(ds.tf.shade(agg, cmap=cc.fire), 'black')

In [ ]:
import numpy as np
import pandas as pd
import datashader as ds
import datashader.transfer_functions as tf
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Generate random geographic data
# n = 100000
# data = pd.DataFrame({
#     'lon': np.random.uniform(-180, 180, n),
#     'lat': np.random.uniform(-90, 90, n)
# })

# Define canvas
canvas = ds.Canvas(plot_width=1000, plot_height=500,
                   x_range=(-180, 180), y_range=(-90, 90))

# Rasterize
# agg = canvas.points(data, 'lon', 'lat')
agg = canvas.points(df, 'longitude', 'latitude')
img = tf.shade(agg, cmap=["lightblue", "darkblue"], how='eq_hist')

# Convert Datashader image to numpy array
img_array = tf.spread(img, px=1).to_pil()

from PIL import Image
img_array = img_array.transpose(Image.FLIP_TOP_BOTTOM)

# Plot with Cartopy
fig = plt.figure(figsize=(15, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Display background map
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, facecolor='white')

# Overlay Datashader image
ax.imshow(img_array, extent=[-180, 180, -90, 90], origin='lower', transform=ccrs.PlateCarree())

plt.title("Random Points on World Map (Datashader + Cartopy)")
plt.show()